# 逻辑回归api介绍
- sklearn.linear_model.LogisticRegression(solver='liblinear', penalty=‘l2’, C = 1.0)

    - solver可选参数:{'liblinear', 'sag', 'saga','newton-cg', 'lbfgs'}，

        - 默认: 'liblinear'；用于优化问题的算法。
        - 对于小数据集来说，“liblinear”是个不错的选择，而“sag”和'saga'对于大型数据集会更快。

        - 对于多类问题，只有'newton-cg'， 'sag'， 'saga'和'lbfgs'可以处理多项损失;“liblinear”仅限于“one-versus-rest”分类。

    - penalty：正则化的种类

    - C：正则化力度

    - 默认将类别数量少的当做正例

LogisticRegression方法相当于 SGDClassifier(loss="log", penalty=" "),SGDClassifier实现了一个普通的随机梯度下降学习。而使用LogisticRegression(实现了SAG)

# 案例：癌症分类预测-良／恶性乳腺癌肿瘤预测

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# 1.获取数据
names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                   'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                   'Normal Nucleoli', 'Mitoses', 'Class']

data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",
                  names=names)
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
# 2.基本数据处理
# 2.1 缺失值处理
data = data.replace(to_replace="?", value=np.NaN)
# dropna()函数返回一个包含非空数据和索引值的Series
data = data.dropna()
# 2.2 确定特征值,目标值
x = data.iloc[:, 1:10]
x.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1


In [4]:
y = data["Class"]
y.head()

0    2
1    2
2    2
3    2
4    2
Name: Class, dtype: int64

In [5]:
# 2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [6]:
# 3.特征工程(标准化)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [7]:
# 4.机器学习(逻辑回归)
estimator = LogisticRegression()
estimator.fit(x_train, y_train)

LogisticRegression()

In [8]:
# 5.模型评估
y_predict = estimator.predict(x_test)
print(y_predict)
estimator.score(x_test, y_test)

[2 4 4 2 2 2 2 2 2 2 2 2 2 4 2 2 4 4 4 2 4 2 4 4 4 2 4 2 2 2 2 2 4 2 2 2 4
 2 2 2 2 4 2 4 4 4 4 2 4 4 2 2 2 2 2 4 2 2 2 2 4 4 4 4 2 4 2 2 4 2 2 2 2 4
 2 2 2 2 2 2 4 4 4 2 4 4 4 4 2 2 2 4 2 4 2 2 2 2 2 2 4 2 2 4 2 2 4 2 4 4 2
 2 2 2 4 2 2 2 2 2 2 4 2 4 2 2 2 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 2 4
 2 2 4 4 4 2 2 4 4 2 4 4 2 2 2 2 2 4 4 2 2 2 4]


0.9766081871345029

# 分类评估报告api
- sklearn.metrics.classification_report(y_true, y_pred, labels=[], target_names=None )
    - y_true：真实目标值
    - y_pred：估计器预测目标值
    - labels:指定类别对应的数字
    - target_names：目标类别名称
    - return：每个类别精确率与召回率
ret = classification_report(y_test, y_predict, labels=(2,4), target_names=("良性", "恶性"))
print(ret)
假设这样一个情况，如果99个样本癌症，1个样本非癌症，不管怎样我全都预测正例(默认癌症为正例),准确率就为99%但是这样效果并不好，这就是样本不均衡下的评估问题

问题：如何衡量样本不均衡下的评估？

In [9]:
from sklearn.metrics import classification_report
ret = classification_report(y_test, y_predict, labels=(2,4), target_names=("良性", "恶性"))
print(ret)

              precision    recall  f1-score   support

          良性       0.98      0.98      0.98       111
          恶性       0.97      0.97      0.97        60

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



精确率(Precision)与召回率(Recall)

精确率：预测结果为正例样本中真实为正例的比例

召回率：真实为正例的样本中预测结果为正例的比例（查得全，对正样本的区分能力）

还有其他的评估标准，F1-score，反映了模型的稳健型

TPR与FPR

TPR = TP / (TP + FN)

所有真实类别为1的样本中，预测类别为1的比例

FPR = FP / (FP + TN)

所有真实类别为0的样本中，预测类别为1的比例

ROC曲线

ROC曲线的横轴就是FPRate，纵轴就是TPRate，当二者相等时，表示的意义则是：对于不论真实类别是1还是0的样本，分类器预测为1的概率是相等的，此时AUC为0.5

说明在不同阈值下分类的好坏

AUC则是综合，AUC高说明在任何阈值 该分类器都效果不错

# AUC指标
- AUC的概率意义是随机取一对正负样本，正样本得分大于负样本的概率
- AUC的最小值为0.5，最大值为1，取值越高越好
- AUC=1，完美分类器，采用这个预测模型时，不管设定什么阈值都能得出完美预测。绝大多数预测的场合，不存在完美分类器。
- 0.5<AUC<1，优于随机猜测。这个分类器（模型）妥善设定阈值的话，能有预测价值。
- 最终AUC的范围在[0.5, 1]之间，并且越接近1越好

# AUC计算API
- from sklearn.metrics import roc_auc_score
- sklearn.metrics.roc_auc_score(y_true, y_score)
- 计算ROC曲线面积，即AUC值
- y_true：每个样本的真实类别，必须为0(反例),1(正例)标记
- y_score：预测得分，可以是正类的估计概率、置信值或者分类器方法的返回值

In [10]:
from sklearn.metrics import roc_auc_score
# 0.5~1之间，越接近于1约好
y_test = np.where(y_test > 2.5, 1, 0)

print("AUC指标：", roc_auc_score(y_test, y_predict))

AUC指标： 0.9743243243243243
